In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('Новая таблица - data.csv')
data.columns = data.columns.str.lower()
data.head()

,client,month,category,device,place,bannertype,querytype (search only),targetingtype,shows,clicks,cost,conversions
0,Псилон,4/1/2021,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Desktop,Networks,cpc_video,none,Phrase,0,0,0,0
1,Псилон,4/1/2021,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Desktop,Networks,cpc_video,none,Retargeting,5949,41,"1666,844364",0
2,Псилон,4/1/2021,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Mobile,Networks,cpc_video,none,Retargeting,191,6,"40,64191525",0
3,Гамма,4/1/2021,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Desktop,Networks,image_ad,none,Retargeting,227,1,"45,74740678",0
4,Гамма,4/1/2021,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Mobile,Networks,image_ad,none,Retargeting,18,0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10478 entries, 0 to 10477
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   client                   10478 non-null  object
 1   month                    10478 non-null  object
 2   category                 10478 non-null  object
 3   device                   10478 non-null  object
 4   place                    10478 non-null  object
 5   bannertype               10478 non-null  object
 6   querytype (search only)  10478 non-null  object
 7   targetingtype            10478 non-null  object
 8   shows                    10478 non-null  int64 
 9   clicks                   10478 non-null  int64 
 10  cost                     10478 non-null  object
 11  conversions              10478 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 982.4+ KB


In [12]:
data[data.duplicated(keep=False)]

,client,month,category,device,place,bannertype,querytype (search only),targetingtype,shows,clicks,cost,conversions,cr,crc


Рекламодатель Сигма жалуется на постоянный рост CPA по своим рекламным кампаниям в последние 4 месяца (с июня по сент 2022). Ваша задача проанализировать его размещение с целью:
1) Выяснить причину растущего CPA
2) Дать рекомендации по снижению и стабилизации CPA до целевого значения (380 руб)

Стоимость обращения (cost per action, CPA) показывает, сколько стоит целевой визит посетителя. CPA — отношение рекламного бюджета к количеству обращений. (затраты на рекламу/количество целевых действий). В этом случае количество целевых действий (визитов) лежит в столбце Conversion. Также известна взаимосвязь CR и CPA. Коэффициент конверсии (CR) — отношение количества целевых визитов к общему количеству визитов, которое, видимо, равное количеству кликов. Чем выше CR, тем дешевле стоит заказ/обращение. 

Оставим только данные за интересующий рекламодателя период:

In [5]:
data['month'] = pd.to_datetime(data['month'], format="%m/%d/%Y")
data = data[(data['month']>'2022-05-01') & (data['month']<'2022-09-01')]

Рассчитаем текущий CPA:

In [6]:
data['cost'] = data['cost'].str.replace(',','.')
data['cost'] = data['cost'].astype(np.float64)
a = data['cost'].sum()
b = data['conversions'].sum()
cpa = a/b
cpa

588.7552274521587

In [8]:
cpa_goal = 380
print("Текущий CPA больше целевого на", cpa/cpa_goal*100-100, "%")

Текущий CPA больше целевого на 54.93558617162071 %


In [ ]:
Рассчитаем CR и CRC:

In [15]:
data['cr'] = data['conversions']/data['clicks']
data['cpc'] = data['cost']/data['clicks']

In [13]:
data[data['cost']==data['cost'].max()]

,client,month,category,device,place,bannertype,querytype (search only),targetingtype,shows,clicks,cost,conversions,cr,crc
6141,Псилон,2022-08-01,КРЕДИТНЫЕ КАРТЫ,Mobile,Networks,text,none,Phrase,334921472,2237376,39766179.12,47259,0.021123,17.773579


In [16]:
data[data['cpc']==data['cpc'].max()]

,client,month,category,device,place,bannertype,querytype (search only),targetingtype,shows,clicks,cost,conversions,cr,crc,cpc
1674,Омега,2022-08-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Desktop,Search,mcbanner,Other,Phrase,8482,3,3589.07189,0,0.0,1196.357297,1196.357297


In [17]:
data['bannertype'].unique()

array(['cpc_video', 'image_ad', 'mcbanner', 'text', 'cpm_banner',
       'dynamic'], dtype=object)

In [22]:
data[data['cr']==data['cr'].min()].sort_values(by='cpc', ascending=False).head()

,client,month,category,device,place,bannertype,querytype (search only),targetingtype,shows,clicks,cost,conversions,cr,crc,cpc
1674,Омега,2022-08-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Desktop,Search,mcbanner,Other,Phrase,8482,3,3589.071890,0,0.0,1196.357297,1196.357297
1670,Сигма,2022-08-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Mobile,Search,mcbanner,Competitor,Phrase,86,1,879.608212,0,0.0,879.608212,879.608212
1678,Псилон,2022-08-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Desktop,Search,mcbanner,Competitor,Phrase,4583,3,2606.679508,0,0.0,868.893169,868.893169
1668,Сигма,2022-08-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Desktop,Search,mcbanner,Other,Phrase,36932,58,47824.885250,0,0.0,824.566987,824.566987
1673,Омега,2022-08-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Desktop,Search,mcbanner,Competitor,Phrase,2967,3,2125.106415,0,0.0,708.368805,708.368805


Предварительно можно выдвинуть гипотезу, что есть проблема с ключевыми фразами для таргетинга, тогда нужно добавлять минус-фразы и дополнить ключевые фразы запросами с высоким CTR.

А больше я не успею))) Но здесь можно сделать групировку и построить графики, посмотреть CR и CPC отдельно по типу таргетинга, типу баннера, устройству, ещё посмотреть динамику по месяцам.